##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regularização de grafos para classificação de sentimentos usando grafos sintetizados

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/neural_structured_learning/tutorials/graph_keras_lstm_imdb"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/neural_structured_learning/tutorials/graph_keras_lstm_imdb.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a>
</td>
</table>

## Visão geral

Este notebook classifica avaliações de filmes como *positivas* ou *negativas*, com base no texto da avaliação. Este é um exemplo de classificação *binária*, um tipo de problema de aprendizado de máquina importante, com diversas aplicações.

Demonstraremos o uso da regularização de grafos neste notebook construindo um grafo a partir da entrada fornecida. A receita geral para construir um modelo regularizado por grafos usando o framework Neural Structured Learning (NSL) quando a entrada não contém um grafo explícito é a seguinte:

1. Crie embeddings para cada amostra de texto na entrada. Isto pode ser feito usando modelos pré-treinados como [word2vec](https://arxiv.org/pdf/1310.4546.pdf), [Swivel](https://arxiv.org/abs/1602.02215), [BERT](https://arxiv.org/abs/1810.04805), etc.
2. Construa um grafo com base nesses embeddings usando uma métrica de similaridade, como distância 'L2', distância 'cosseno', etc. Os nós no grafo correspondem a amostras e as arestas no grafo correspondem à similaridade entre pares de amostras.
3. Gere dados de treinamento a partir do grafo sintetizado acima e de características de amostra. Os dados de treinamento resultantes conterão características vizinhas além das características originais do nó.
4. Crie uma rede neural como modelo de referência usando a API sequencial, funcional ou de subclasse do <code>Keras</code>.
5. Envolva o modelo base com a classe wrapper <strong><code>GraphRegularization</code></strong>, que é fornecida pelo framework NSL, para criar um novo modelo de grafo <code>Keras</code>. Este novo modelo incluirá uma perda de regularização do grafo como termo de regularização em seu objetivo de treinamento.
6. Treine e avalie o modelo do grafo <code>Keras</code>.

**Observação**: Esperamos que os leitores levem cerca de 1 hora para concluir este tutorial.

## Requisitos

1. Instale o pacote Neural Structured Learning.
2. Instale o tensorflow-hub.

In [ ]:
!pip install --quiet neural-structured-learning
!pip install --quiet tensorflow-hub

## Dependências e importações

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import neural_structured_learning as nsl

import tensorflow as tf
import tensorflow_hub as hub

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## Dataset IMDB

O [dataset do IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) contém o texto de 50.000 avaliações de filmes do [Internet Movie Database](https://www.imdb.com/) . Elas são divididas em 25.000 avaliações para treinamento e 25.000 avaliações para testes. Os datasets de treinamento e teste são *balanceados*, o que significa que contêm um número igual de avaliações positivas e negativas.

Neste tutorial, usaremos uma versão pré-processada do dataset IMDB.

### Baixe o dataset do IMDB pré-processado

O dataset IMDB vem com o TensorFlow. Já foi pré-processado de forma que as avaliações (sequências de palavras) foram convertidas em sequências de inteiros, onde cada inteiro representa uma palavra específica num dicionário.

O código a seguir baixa o dataset IMDB (ou usa uma cópia em cache se já tiver sido baixado):

In [ ]:
imdb = tf.keras.datasets.imdb
(pp_train_data, pp_train_labels), (pp_test_data, pp_test_labels) = (
    imdb.load_data(num_words=10000))

O argumento `num_words=10000` mantém as 10.000 palavras que ocorrem com mais frequência nos dados de treinamento. As palavras raras são descartadas para manter o tamanho do vocabulário controlável.

### Explore os dados

Vamos dedicar um momento para entender o formato dos dados. O dataset vem pré-processado: cada exemplo é um array de números inteiros que representa as palavras da crítica do filme. Cada rótulo é um valor inteiro de 0 ou 1, onde 0 é uma avaliação negativa e 1 é uma avaliação positiva.

In [ ]:
print('Training entries: {}, labels: {}'.format(
    len(pp_train_data), len(pp_train_labels)))
training_samples_count = len(pp_train_data)

O texto das avaliações foi convertido para números inteiros, onde cada número inteiro representa uma palavra específica num dicionário. Eis como se parece a primeira avaliação:

In [ ]:
print(pp_train_data[0])

As avaliações de filmes podem ter comprimentos diferentes. O código abaixo mostra o número de palavras na primeira e na segunda avaliações. Como as entradas de uma rede neural devem ter o mesmo comprimento, vamos ter que resolver isso mais tarde.

In [ ]:
len(pp_train_data[0]), len(pp_train_data[1])

### Converta os inteiros de volta para palavras

Pode ser útil saber como converter números inteiros de volta ao texto correspondente. Aqui, criaremos uma função helper para pesquisar um objeto de dicionário que contém o mapeamento de número inteiro para string:

In [ ]:
def build_reverse_word_index():
  # A dictionary mapping words to an integer index
  word_index = imdb.get_word_index()

  # The first indices are reserved
  word_index = {k: (v + 3) for k, v in word_index.items()}
  word_index['<PAD>'] = 0
  word_index['<START>'] = 1
  word_index['<UNK>'] = 2  # unknown
  word_index['<UNUSED>'] = 3
  return dict((value, key) for (key, value) in word_index.items())

reverse_word_index = build_reverse_word_index()

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

Agora podemos usar a função `decode_review` para exibir o texto da primeira avaliação:

In [ ]:
decode_review(pp_train_data[0])

## Construção do grafo

A construção de grafos envolve a criação de embeddings para amostras de texto e, em seguida, o uso de uma função de similaridade para comparar os embeddings.

Antes de prosseguir, primeiro criamos um diretório para armazenar os artefatos criados por este tutorial.

In [ ]:
!mkdir -p /tmp/imdb

### Crie embeddings de amostra

Usaremos embeddings Swivel pré-treinados para criar embeddings no formato `tf.train.Example` para cada amostra na entrada. Armazenaremos os embeddings resultantes no formato `TFRecord` junto com uma característica adicional que representa o ID de cada amostra. Isto é importante e nos permitirá posteriormente combinar os embeddings de amostra com os nós correspondentes no grafo.

In [ ]:
pretrained_embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'

hub_layer = hub.KerasLayer(
    pretrained_embedding, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
def _int64_feature(value):
  """Returns int64 tf.train.Feature."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value.tolist()))


def _bytes_feature(value):
  """Returns bytes tf.train.Feature."""
  return tf.train.Feature(
      bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))


def _float_feature(value):
  """Returns float tf.train.Feature."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value.tolist()))


def create_embedding_example(word_vector, record_id):
  """Create tf.Example containing the sample's embedding and its ID."""

  text = decode_review(word_vector)

  # Shape = [batch_size,].
  sentence_embedding = hub_layer(tf.reshape(text, shape=[-1,]))

  # Flatten the sentence embedding back to 1-D.
  sentence_embedding = tf.reshape(sentence_embedding, shape=[-1])

  features = {
      'id': _bytes_feature(str(record_id)),
      'embedding': _float_feature(sentence_embedding.numpy())
  }
  return tf.train.Example(features=tf.train.Features(feature=features))


def create_embeddings(word_vectors, output_path, starting_record_id):
  record_id = int(starting_record_id)
  with tf.io.TFRecordWriter(output_path) as writer:
    for word_vector in word_vectors:
      example = create_embedding_example(word_vector, record_id)
      record_id = record_id + 1
      writer.write(example.SerializeToString())
  return record_id


# Persist TF.Example features containing embeddings for training data in
# TFRecord format.
create_embeddings(pp_train_data, '/tmp/imdb/embeddings.tfr', 0)

### Construa um grafo

Agora que temos os embeddings das amostras, podemos utilizá-los para construir um grafo de similaridade, ou seja, os nós neste grafo corresponderão a amostras e as arestas do grafo corresponderão à similaridade entre pares de nós.

O Neural Structured Learning fornece uma biblioteca de construção de grafos para construir um grafo com base em embeddings de amostras. Ele usa <strong>similaridade de cosseno</strong> como medida de similaridade para comparar embeddings e construir arestas entre eles. Também nos permite especificar um limiar de similaridade, que pode ser usado para descartar arestas diferentes do grafo final. Neste exemplo, usando 0,99 como limiar de similaridade, obtemos um grafo que possui 429.415 arestas bidirecionais. Aqui estamos usando o suporte do construtor de grafos para [hashing sensível à localidade](https://en.wikipedia.org/wiki/Locality-sensitive_hashing) (LSH) para acelerar a construção de grafos. Para detalhes sobre como usar o suporte LSH do construtor de grafos, consulte a documentação da API [`build_graph_from_config`](https://www.tensorflow.org/neural_structured_learning/api_docs/python/nsl/tools/build_graph_from_config).

In [ ]:
graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=0.99, lsh_splits=32, lsh_rounds=15, random_seed=12345)
nsl.tools.build_graph_from_config(['/tmp/imdb/embeddings.tfr'],
                                  '/tmp/imdb/graph_99.tsv',
                                  graph_builder_config)

Cada aresta bidirecional é representada por duas arestas direcionadas no arquivo TSV de saída, de modo que o arquivo contém 429.415 * 2 = 858.830 linhas no total:

In [ ]:
!wc -l /tmp/imdb/graph_99.tsv

**Observação:** A qualidade do grafo e, por extensão, a qualidade dos embeddings, são muito importantes para a regularização do grafo. Embora usemos embeddings Swivel neste notebook, usar embeddings BERT, por exemplo, provavelmente capturará a semântica da avaliação com mais exatidão. Incentivamos os usuários a usarem os embeddings de sua escolha e conforme seja apropriado às suas necessidades.

## Características de amostra

Criamos características de amostra para nosso problema usando o formato `tf.train.Example` e os persistimos no formato `TFRecord`. Cada amostra incluirá as três características a seguir:

1. **id**: o ID do nó da amostra.
2. **words**: uma lista int64 contendo IDs de palavras.
3. **label**: um singleton int64 que identifica a classe alvo da avaliação.

In [ ]:
def create_example(word_vector, label, record_id):
  """Create tf.Example containing the sample's word vector, label, and ID."""
  features = {
      'id': _bytes_feature(str(record_id)),
      'words': _int64_feature(np.asarray(word_vector)),
      'label': _int64_feature(np.asarray([label])),
  }
  return tf.train.Example(features=tf.train.Features(feature=features))

def create_records(word_vectors, labels, record_path, starting_record_id):
  record_id = int(starting_record_id)
  with tf.io.TFRecordWriter(record_path) as writer:
    for word_vector, label in zip(word_vectors, labels):
      example = create_example(word_vector, label, record_id)
      record_id = record_id + 1
      writer.write(example.SerializeToString())
  return record_id

# Persist TF.Example features (word vectors and labels) for training and test
# data in TFRecord format.
next_record_id = create_records(pp_train_data, pp_train_labels,
                                '/tmp/imdb/train_data.tfr', 0)
create_records(pp_test_data, pp_test_labels, '/tmp/imdb/test_data.tfr',
               next_record_id)

## Aumente os dados de treinamento com vizinhos de grafo

Como temos as características de amostra e o grafo sintetizado, podemos gerar os dados de treinamento aumentados para o Neural Structured Learning. O framework NSL fornece uma biblioteca para combinar o grafo e as características de amostra para produzir os dados de treinamento finais para regularização do grafo. Os dados de treinamento resultantes incluirão características de amostra originais, bem como características de seus vizinhos correspondentes.

Neste tutorial, consideramos arestas não direcionadas e usamos no máximo 3 vizinhos por amostra para aumentar os dados de treinamento com vizinhos do grafo.

In [ ]:
nsl.tools.pack_nbrs(
    '/tmp/imdb/train_data.tfr',
    '',
    '/tmp/imdb/graph_99.tsv',
    '/tmp/imdb/nsl_train_data.tfr',
    add_undirected_edges=True,
    max_nbrs=3)

## Modelo de referência

Agora estamos prontos para construir um modelo base sem regularização de grafos. Para construir este modelo, podemos usar embeddings que foram usados ​​na construção do grafo, ou podemos aprender novos embeddings juntamente com a tarefa de classificação. Para os fins deste notebook, faremos o último.

### Variáveis ​​globais

In [ ]:
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'

### Hiperparâmetros

Usaremos uma instância de `HParams` para incluir vários hiperparâmetros e constantes usados ​​para treinamento e avaliação. Descrevemos brevemente cada um deles abaixo:

- **num_classes**: há 2 classes – *positive* e *negative*.

- **max_seq_length**: é o número máximo de palavras consideradas em cada avaliação de filme neste exemplo.

- **vocab_size**: é o tamanho do vocabulário considerado neste exemplo.

- **distance_type**: é a métrica de distância usada para regularizar a amostra com seus vizinhos.

- **graph_regularization_multiplier**: controla o peso relativo do termo de regularização do grafo na função de perda geral.

- **num_neighbors**: o número de vizinhos usados ​​para regularização do grafo. Este valor deve ser menor ou igual ao argumento `max_nbrs` usado acima ao invocar `nsl.tools.pack_nbrs`.

- **num_fc_units**: o número de unidades na camada totalmente conectada da rede neural.

- **train_epochs**: o número de épocas de treinamento.

- **batch_size**: tamanho do lote usado para treinamento e avaliação.

- **eval_steps**: o número de lotes a serem processados ​​antes que a avaliação seja concluída. Se definido como `None`, todas as instâncias no dataset de testes serão avaliadas.

In [ ]:
class HParams(object):
  """Hyperparameters used for training."""
  def __init__(self):
    ### dataset parameters
    self.num_classes = 2
    self.max_seq_length = 256
    self.vocab_size = 10000
    ### neural graph learning parameters
    self.distance_type = nsl.configs.DistanceType.L2
    self.graph_regularization_multiplier = 0.1
    self.num_neighbors = 2
    ### model architecture
    self.num_embedding_dims = 16
    self.num_lstm_dims = 64
    self.num_fc_units = 64
    ### training parameters
    self.train_epochs = 10
    self.batch_size = 128
    ### eval parameters
    self.eval_steps = None  # All instances in the test set are evaluated.

HPARAMS = HParams()

### Preparação dos dados

As avaliações – os arrays de inteiros – devem ser convertidas em tensores antes de serem alimentadas na rede neural. Essa conversão pode ser feita de duas maneiras:

- Converta os arrays em vetores de valores `0` e `1` indicando a ocorrência de palavras, semelhante a uma one-hot encoding. Por exemplo, a sequência `[3, 5]` se tornaria um vetor de `10000` dimensões que é todo zero, exceto os índices `3` e `5`, que serão 1. Em seguida, torne esta a primeira camada em nossa rede – uma camada `Dense` – que poderá lidar com dados vetoriais em ponto flutuante. Essa abordagem consome muita memória, exigindo uma matriz de tamanho `num_words * num_reviews`.

- Como alternativa, podemos preencher os arrays para que todos tenham o mesmo comprimento e, em seguida, criar um tensor inteiro de forma `max_length * num_reviews`. Podemos usar uma camada de embedding capaz de lidar com esse formato como a primeira camada da nossa rede.

Neste tutorial, usaremos a segunda abordagem.

Como as avaliações de filmes devem ter a mesma duração, usaremos a função `pad_sequence` definida abaixo para padronizar as durações.

In [ ]:
def make_dataset(file_path, training=False):
  """Creates a `tf.data.TFRecordDataset`.

  Args:
    file_path: Name of the file in the `.tfrecord` format containing
      `tf.train.Example` objects.
    training: Boolean indicating if we are in training mode.

  Returns:
    An instance of `tf.data.TFRecordDataset` containing the `tf.train.Example`
    objects.
  """

  def pad_sequence(sequence, max_seq_length):
    """Pads the input sequence (a `tf.SparseTensor`) to `max_seq_length`."""
    pad_size = tf.maximum([0], max_seq_length - tf.shape(sequence)[0])
    padded = tf.concat(
        [sequence.values,
         tf.fill((pad_size), tf.cast(0, sequence.dtype))],
        axis=0)
    # The input sequence may be larger than max_seq_length. Truncate down if
    # necessary.
    return tf.slice(padded, [0], [max_seq_length])

  def parse_example(example_proto):
    """Extracts relevant fields from the `example_proto`.

    Args:
      example_proto: An instance of `tf.train.Example`.

    Returns:
      A pair whose first value is a dictionary containing relevant features
      and whose second value contains the ground truth labels.
    """
    # The 'words' feature is a variable length word ID vector.
    feature_spec = {
        'words': tf.io.VarLenFeature(tf.int64),
        'label': tf.io.FixedLenFeature((), tf.int64, default_value=-1),
    }
    # We also extract corresponding neighbor features in a similar manner to
    # the features above during training.
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, i,
                                         NBR_WEIGHT_SUFFIX)
        feature_spec[nbr_feature_key] = tf.io.VarLenFeature(tf.int64)

        # We assign a default value of 0.0 for the neighbor weight so that
        # graph regularization is done on samples based on their exact number
        # of neighbors. In other words, non-existent neighbors are discounted.
        feature_spec[nbr_weight_key] = tf.io.FixedLenFeature(
            [1], tf.float32, default_value=tf.constant([0.0]))

    features = tf.io.parse_single_example(example_proto, feature_spec)

    # Since the 'words' feature is a variable length word vector, we pad it to a
    # constant maximum length based on HPARAMS.max_seq_length
    features['words'] = pad_sequence(features['words'], HPARAMS.max_seq_length)
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        features[nbr_feature_key] = pad_sequence(features[nbr_feature_key],
                                                 HPARAMS.max_seq_length)

    labels = features.pop('label')
    return features, labels

  dataset = tf.data.TFRecordDataset([file_path])
  if training:
    dataset = dataset.shuffle(10000)
  dataset = dataset.map(parse_example)
  dataset = dataset.batch(HPARAMS.batch_size)
  return dataset


train_dataset = make_dataset('/tmp/imdb/nsl_train_data.tfr', True)
test_dataset = make_dataset('/tmp/imdb/test_data.tfr')

### Criação do modelo

Uma rede neural é criada empilhando-se camadas. Isto requer duas decisões de arquitetura principais:

- Quantas camadas usar no modelo?
- Quantas *unidades ocultas* usar em cada camada?

Neste exemplo, os dados de entrada consistem num array de índices de palavras. Os rótulos a serem previstos são 0 ou 1.

Usaremos um LSTM bidirecional como nosso modelo de referência neste tutorial.

In [ ]:
# This function exists as an alternative to the bi-LSTM model used in this
# notebook.
def make_feed_forward_model():
  """Builds a simple 2 layer feed forward neural network."""
  inputs = tf.keras.Input(
      shape=(HPARAMS.max_seq_length,), dtype='int64', name='words')
  embedding_layer = tf.keras.layers.Embedding(HPARAMS.vocab_size, 16)(inputs)
  pooling_layer = tf.keras.layers.GlobalAveragePooling1D()(embedding_layer)
  dense_layer = tf.keras.layers.Dense(16, activation='relu')(pooling_layer)
  outputs = tf.keras.layers.Dense(1)(dense_layer)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


def make_bilstm_model():
  """Builds a bi-directional LSTM model."""
  inputs = tf.keras.Input(
      shape=(HPARAMS.max_seq_length,), dtype='int64', name='words')
  embedding_layer = tf.keras.layers.Embedding(HPARAMS.vocab_size,
                                              HPARAMS.num_embedding_dims)(
                                                  inputs)
  lstm_layer = tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(HPARAMS.num_lstm_dims))(
          embedding_layer)
  dense_layer = tf.keras.layers.Dense(
      HPARAMS.num_fc_units, activation='relu')(
          lstm_layer)
  outputs = tf.keras.layers.Dense(1)(dense_layer)
  return tf.keras.Model(inputs=inputs, outputs=outputs)


# Feel free to use an architecture of your choice.
model = make_bilstm_model()
model.summary()

As camadas são empilhadas sequencialmente para construir o classificador:

1. A primeira camada é uma camada `Input` que utiliza o vocabulário codificado por inteiros.
2. A camada seguinte é uma camada `Embedding`, que pega o vocabulário codificado por inteiros e procura o vetor de incorporação para cada índice de palavras. Esses vetores adicionam uma dimensão ao array de saída. As dimensões resultantes são: `(batch, sequence, embedding)`.
3. Em seguida, uma camada LSTM bidirecional retorna um vetor de saída de comprimento fixo para cada exemplo.
4. O vetor de saída com tamanho fixo é passado através de uma camada (`Dense`) totalmente conectada com 64 unidades ocultas.
5. A última camada está densamente conectada a um único nó de saída. Usando a função de ativação `sigmoid`, esse valor é um valor flutuante entre 0 e 1, representando uma probabilidade ou nível de confiança.

### Unidades ocultas

O modelo acima possui duas camadas intermediárias ou "ocultas", entre a entrada e a saída, e excluindo a camada `Embedding`. O número de saídas (unidades, nós ou neurônios) é a dimensão do espaço representacional da camada. Em outras palavras, a quantidade de liberdade permitida à rede ao aprender uma representação interna.

Se um modelo tiver mais unidades ocultas (um espaço de representação de dimensão superior) e/ou mais camadas, a rede poderá aprender representações mais complexas. No entanto, isso deixa a rede mais cara do ponto de vista computacional e pode levar ao aprendizado de padrões indesejados – padrões que melhoram o desempenho nos dados de treinamento, mas não nos dados de teste. Isto é chamado de *overfitting*.

### Função de perda e otimizador

Um modelo precisa de uma função de perda e um otimizador para o treinamento. Como este é um problema de classificação binária e o modelo gera como saída uma probabilidade (uma camada de unidade única com uma ativação sigmóide), usaremos a função de perda `binary_crossentropy`.

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Crie um conjunto de validação

Durante o treinamento, queremos verificar a precisão do modelo em dados que ele não viu antes. Crie um *dataset de validação* separando uma fração dos dados de treinamento originais. (Por que não usar o dataset de testes agora? Nosso objetivo é desenvolver e ajustar nosso modelo usando apenas os dados de treinamento e, em seguida, usar os dados de teste apenas uma vez para avaliar nossa exatidão).

Neste tutorial, consideramos aproximadamente 10% das amostras de treinamento inicial (10% de 25.000) como dados rotulados para treinamento e o restante como dados de validação. Como a divisão inicial de treinamento/teste foi 50/50 (25.000 amostras cada), a divisão efetiva de treinamento/validação/teste que temos agora é 5/45/50.

Observe que 'train_dataset' já foi agrupado e embaralhado. 

In [ ]:
validation_fraction = 0.9
validation_size = int(validation_fraction *
                      int(training_samples_count / HPARAMS.batch_size))
print(validation_size)
validation_dataset = train_dataset.take(validation_size)
train_dataset = train_dataset.skip(validation_size)

### Treine o modelo

Treine o modelo em minilotes. Durante o treinamento, monitore a perda e a precisão do modelo no dataset de validação:

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=HPARAMS.train_epochs,
    verbose=1)

### Avalie o modelo

Vamos conferir o desempenho do modelo. Serão retornados dois valores: perda (um número que representa o erro; quanto menor, melhor) e exatidão.

In [ ]:
results = model.evaluate(test_dataset, steps=HPARAMS.eval_steps)
print(results)

### Crie um grafo de precisão/perda ao longo do tempo

`model.fit()` retorna um objeto `History` que contém um dicionário com tudo o que aconteceu durante o treinamento:

In [ ]:
history_dict = history.history
history_dict.keys()

Há quatro entradas: uma para cada métrica monitorada durante o treinamento e a validação. Você usará esses valores para plotar a perda do treinamento e da validação para fins comparativos, além da exatidão do treinamento e da validação:

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "-r^" is for solid red line with triangle markers.
plt.plot(epochs, loss, '-r^', label='Training loss')
# "-b0" is for solid blue line with circle markers.
plt.plot(epochs, val_loss, '-bo', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, '-r^', label='Training acc')
plt.plot(epochs, val_acc, '-bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.show()

Observe que a perda do treinamento *diminui* a cada época, e a exatidão do treinamento *aumenta* a cada época. Isso é o esperado ao usar uma otimização do método do gradiente descendente, que deve minimizar a quantidade desejada em cada iteração.

## Regularização de grafos

Agora estamos prontos para tentar a regularização de grafos usando o modelo base que construímos acima. Usaremos a classe wrapper `GraphRegularization` fornecida pela framework Neural Structured Learning para encapsular o modelo de referência (bi-LSTM) para incluir a regularização do grafo. As demais etapas para treinar e avaliar o modelo regularizado por grafos são semelhantes às do modelo de referência.

### Crie um modelo regularizado por grafo

Para avaliar o benefício incremental da regularização de grafos, criaremos uma nova instância do modelo de referência. Isto é necessário porque `model` já foi treinado por algumas iterações, e reutilizar esse modelo treinado para criar um modelo regularizado por grafo não produziria uma comparação justa para `model`.

In [ ]:
# Build a new base LSTM model.
base_reg_model = make_bilstm_model()

In [ ]:
# Wrap the base model with graph regularization.
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=HPARAMS.num_neighbors,
    multiplier=HPARAMS.graph_regularization_multiplier,
    distance_type=HPARAMS.distance_type,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_reg_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Treine o modelo

In [ ]:
graph_reg_history = graph_reg_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=HPARAMS.train_epochs,
    verbose=1)

### Avalie o modelo

In [ ]:
graph_reg_results = graph_reg_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)
print(graph_reg_results)

### Crie um grafo de precisão/perda ao longo do tempo

In [ ]:
graph_reg_history_dict = graph_reg_history.history
graph_reg_history_dict.keys()

Há cinco entradas no total no dicionário: perda de treinamento, precisão de treinamento, perda de grafo de treinamento, perda de validação e exatidão de validação. Podemos representá-los todos juntos para comparação. Observe que a perda do grafo só é calculada durante o treinamento.

In [ ]:
acc = graph_reg_history_dict['accuracy']
val_acc = graph_reg_history_dict['val_accuracy']
loss = graph_reg_history_dict['loss']
graph_loss = graph_reg_history_dict['scaled_graph_loss']
val_loss = graph_reg_history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.clf()   # clear figure

# "-r^" is for solid red line with triangle markers.
plt.plot(epochs, loss, '-r^', label='Training loss')
# "-gD" is for solid green line with diamond markers.
plt.plot(epochs, graph_loss, '-gD', label='Training graph loss')
# "-b0" is for solid blue line with circle markers.
plt.plot(epochs, val_loss, '-bo', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, '-r^', label='Training acc')
plt.plot(epochs, val_acc, '-bo', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

plt.show()

## O poder da aprendizagem semissupervisionada

O aprendizado semissupervisionado e, mais especificamente, a regularização de grafos no contexto deste tutorial, pode ser muito poderoso quando a quantidade de dados de treinamento é pequena. A falta de dados de treinamento é compensada pelo aproveitamento da similaridade entre as amostras de treinamento, o que não é possível no aprendizado supervisionado tradicional.

Definimos a ***taxa de supervisão*** como a proporção entre amostras de treinamento e o número total de amostras que inclui amostras de treinamento, validação e teste. Neste notebook, usamos uma taxa de supervisão de 0,05 (ou seja, 5% dos dados rotulados) para treinar tanto o modelo de referência quanto o modelo regularizado por grafo. Ilustramos o impacto da taxa de supervisão na exatidão do modelo na célula abaixo.

In [ ]:
# Accuracy values for both the Bi-LSTM model and the feed forward NN model have
# been precomputed for the following supervision ratios.

supervision_ratios = [0.3, 0.15, 0.05, 0.03, 0.02, 0.01, 0.005]

model_tags = ['Bi-LSTM model', 'Feed Forward NN model']
base_model_accs = [[84, 84, 83, 80, 65, 52, 50], [87, 86, 76, 74, 67, 52, 51]]
graph_reg_model_accs = [[84, 84, 83, 83, 65, 63, 50],
                        [87, 86, 80, 75, 67, 52, 50]]

plt.clf()  # clear figure

fig, axes = plt.subplots(1, 2)
fig.set_size_inches((12, 5))

for ax, model_tag, base_model_acc, graph_reg_model_acc in zip(
    axes, model_tags, base_model_accs, graph_reg_model_accs):

  # "-r^" is for solid red line with triangle markers.
  ax.plot(base_model_acc, '-r^', label='Base model')
  # "-gD" is for solid green line with diamond markers.
  ax.plot(graph_reg_model_acc, '-gD', label='Graph-regularized model')
  ax.set_title(model_tag)
  ax.set_xlabel('Supervision ratio')
  ax.set_ylabel('Accuracy(%)')
  ax.set_ylim((25, 100))
  ax.set_xticks(range(len(supervision_ratios)))
  ax.set_xticklabels(supervision_ratios)
  ax.legend(loc='best')

plt.show()

Pode-se observar que à medida que o índice de supervisão diminui, a exatidão do modelo também diminui. Isso é verdade tanto para o modelo de referência quanto para o modelo regularizado por grafos, independentemente da arquitetura do modelo utilizada. No entanto, observe que o modelo regularizado por grafos tem desempenho melhor que o modelo de referência para ambas as arquiteturas. Em particular, para o modelo Bi-LSTM, quando o índice de supervisão é 0,01, a exatidão do modelo regularizado por grafos é **~20%** maior que a do modelo de referência. Isso se deve principalmente ao aprendizado semissupervisionado para o modelo regularizado por grafos, onde a similaridade estrutural entre as amostras de treinamento é usada além das próprias amostras de treinamento.

## Conclusão

Demonstramos o uso da regularização de grafos usando o framework Neural Structured Learning (NSL) mesmo quando a entrada não contém um grafo explícito. Consideramos a tarefa de classificação de sentimentos de avaliações de filmes do IMDB, para a qual sintetizamos um grafo de similaridade baseado em embeddings de avaliações. Encorajamos os usuários a experimentar mais variando hiperparâmetros, alterando a quantidade de supervisão e usando diferentes arquiteturas de modelo.